In [1]:
from numpy import sqrt
from numpy import loadtxt
import numpy as np
from pandas import read_csv
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt


dataset = loadtxt('pv_large_4d_sample.csv', delimiter=',')
feature_names = ['d1', 'd2', 'd3', 'd4', 'eta']


#df.columns = feature_names
#print(df.head())

#df = df.rename(columns={'eta': 'efficiency'})
#print(df.describe())


#Split into features and target (Price)
#X = df.drop('efficiency', axis = 1)
#y = df['efficiency']
X = dataset[:,0:4]
y = dataset[:,4]

# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
######################################################################

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

############################################################
### Decision tree
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)




DecisionTreeRegressor()

In [3]:
dataset2 = loadtxt('pv_4_layer.csv', delimiter=',')

X_test = dataset2[:,0:4]/200.
y_expected = dataset2[:,4]

predictions = tree.predict(X_test)
error=0.
print(len(y_expected))
idx = 0
for i in range(1, 199):
    for j in range(1, 199):
        idx_i_f = (i+1)*200 + j
        idx_i_b = (i-1)*200 + j
        idx_j_f = i*200 + j + 1
        idx_j_b = i*200 + j - 1
        real_grad_j = (y_expected[idx_j_f] - y_expected[idx_j_b])/2e-9
        real_grad_i = (y_expected[idx_i_f] - y_expected[idx_i_b])/2e-9
        norm = np.sqrt(real_grad_j **2 + real_grad_i ** 2)
        nn_grad_j = (predictions[idx_j_f] - predictions[idx_j_b])/2e-9
        nn_grad_i = (predictions[idx_i_f] - predictions[idx_i_b])/2e-9
        error += np.sqrt( (real_grad_j - nn_grad_j)**2 + (real_grad_i - nn_grad_i)**2 ) / norm
        
#print(len(y_expected))
#for i in range(0, len(y_expected)):
#    error += np.abs(predictions[i]-y_expected[i])
print(error/len(y_expected))


40000
1.1522938367723938


In [135]:
##############################################
#Random forest.
#Increase number of tress and see the effect
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators = 30, random_state=30)
model.fit(X_train, y_train)

#y_pred_RF = model.predict(X_test)
predictions = model.predict(X_test)
base_str = "4 49 "
f = open('rf_pv_4_layer_4d.dat', 'w')
for i in range(0, 200):
    for j in range(0, 200):
        idx = i*200+j
        print_str = base_str + str(X_test[idx,2]*200) + " " + str(X_test[idx,3]*200) + " " 
        p_str = str(predictions[idx])
        print_str += p_str[1:8] + "\n"
        f.write(print_str)
    f.write("\n")
    
f.close()
#mse_RF = mean_squared_error(y_test, y_pred_RF)
#mae_RF = mean_absolute_error(y_test, y_pred_RF)
#print('Mean squared error using Random Forest: ', mse_RF)
#print('Mean absolute error Using Random Forest: ', mae_RF)



##Feature ranking...
#import pandas as pd
#feature_list = list(X.columns)
#feature_imp = pd.Series(model.feature_importances_, index=feature_list).sort_values(ascending=False)
#print(feature_imp)